# Convert `.tml` Varian print template file to `.json`

In [171]:
from pathlib import Path
import re


In [172]:
base_path = Path.cwd()
file_name = 'AllVariables.tml'

file = base_path / file_name

In [173]:
raw_template = file.read_text(encoding='UTF8')

In [178]:
regex_patterns = [
    # Remove lines containing formatting instructions
    (r'Description=.+$', ''),
    (r'PaperSize=.+$', ''),
    (r'PaperOrientation=.+$', ''),
    (r'TextRight=.+$', '',),
    (r'Height=.+$', ''),
    (r'PageBreak=.+$', ''),
    (r'Section=.+$', ''),
    (r'Font=.+$', ''),
    (r'HorizontalLine=.+$', ''),
    (r'Margin=.+$', ''),
    (r'RelativeMove=.+$', ''),
    (r'RelativeMoveText=.+$', ''),
    (r'Move=.+$', ''),
    # Remove any DVH Graph
    (r'DVHGraph=.+$', ''),
    # Convert lines with the format: Text=*<GROUP NAME>* VARIABLES
    # to Json objects that contain objects:
    # },                       # Ends previous object group
    # "*<GROUP NAME>*": {      # Starts a new group 
    (r'Text=([A-Za-z ]+) (VARIABLES|Variables)([\r\n]+)', r'},\3"\1": {\3'),
    # convert lines with the format: Text=$$ *<VariableName>* : $*<VariableName>*
    # to Json objects: "*<VariableName>*":, $*<VariableName>*,
    (r'Text=[$]{2}[ ]*([A-Za-z_0-9$#]+)[ ]*:[ ]*([$][A-Za-z_0-9$#]+)', r'"\1": "\2",'),
    # Convert lines within a loop to an array:
    # convert lines with the format: Loop=*<VariableName>*
    # to a Json array of objects:
    # *<indent>*"*<VariableName>*": [{
    # embed original Loop command from tml inside Json array structure.     
    # Loop=*<VariableName>*   
    (r'([ ]*)Loop=([A-Za-z_0-9]+)([\r\n]+)', r'\1"\2": [{\3\1Loop=\2\3\1'),
    # Add end-of array Json syntax after an EndLoop  command:
    # convert lines with the format: Loop=*<VariableName>*
    # to a Json array of objects:
    # *<indent>*}
    # *<indent>*EndLoop
    # *<indent>*],
    (r'([ ]*)EndLoop([\r\n]+)', r'\1},\2\1EndLoop\2\1],\2'),
    # Remove comment lines containing 'Text='
    (r'Text=.+$', ''),    
    # Remove comments (everything on a line after '#', 
    # except when '#' is preceded by '$')
    (r'#(?<!$#)[^\r\n]*$', ''),
    # Remove contents of lines that contain only spaces.
    (r'^[ ]+$', '')    
    ]

In [179]:
#print(raw_template)
#raw_template

In [180]:
mod_str = raw_template
for ptrn, repl in regex_patterns:
    mod_str, rep_num = re.subn(ptrn, repl, mod_str, flags=re.MULTILINE)
    print(f'Replaced {rep_num} instances of {ptrn}')
    
    
# Remove blank lines
while '\n\n' in mod_str:
    mod_str = mod_str.replace('\n\n', '\n')
# Replace the first }, with {
mod_str = mod_str.replace('},', '{')
# Add } to the end of the text
mod_str = mod_str + '}'

Replaced 1 instances of Description=.+$
Replaced 1 instances of PaperSize=.+$
Replaced 1 instances of PaperOrientation=.+$
Replaced 1 instances of TextRight=.+$
Replaced 2 instances of Height=.+$
Replaced 2 instances of PageBreak=.+$
Replaced 4 instances of Section=.+$
Replaced 58 instances of Font=.+$
Replaced 20 instances of HorizontalLine=.+$
Replaced 72 instances of Margin=.+$
Replaced 35 instances of RelativeMove=.+$
Replaced 1 instances of RelativeMoveText=.+$
Replaced 76 instances of Move=.+$
Replaced 2 instances of DVHGraph=.+$
Replaced 28 instances of Text=([A-Za-z ]+) (VARIABLES|Variables)([\r\n]+)
Replaced 426 instances of Text=[$]{2}[ ]*([A-Za-z_0-9$#]+)[ ]*:[ ]*([$][A-Za-z_0-9$#]+)
Replaced 20 instances of ([ ]*)Loop=([A-Za-z_0-9]+)([\r\n]+)
Replaced 20 instances of ([ ]*)EndLoop([\r\n]+)
Replaced 113 instances of Text=.+$
Replaced 6 instances of #(?<!$#)[^\r\n]*$
Replaced 174 instances of ^[ ]+$


In [184]:
command_text = [
    'Loop',
    'EndLoop',
    'If',
    'EndIf'
    ]
pt = (r'([\r\n]+[^\r\n]*)(EndLoop)([^\r\n]*[\r\n]+)', r'\1},\2\1EndLoop\2\1],\2'),

In [185]:
print(mod_str)


{
"PLAN SUM": {
"PlanSumId": "$PlanSumId",
"PlanSumName": "$PlanSumName",
"PlanSumCourseId": "$PlanSumCourseId",
"SumPlans": [{
Loop=SumPlans
  "PlanIdOfSum": "$PlanIdOfSum",
  "PlanNameOfSum": "$PlanNameOfSum",
  "DosePerFrac_ForSum": "$DosePerFrac_ForSum",
  "NbrOfFrac_ForSum": "$NbrOfFrac_ForSum",
  "TotalDose_ForSum": "$TotalDose_ForSum",
  "TotalDoseAtPrimary_ForSum": "$TotalDoseAtPrimary_ForSum",
  "DosePerFraction_ForSum": "$DosePerFraction_ForSum",
{
EndLoop
],
{
"GENERAL": {
"ApplicationName": "$ApplicationName",
"ApplicationVersion": "$ApplicationVersion",
"TaskName": "$TaskName",
"TaskVersion": "$TaskVersion",
"Comment": "$Comment",
"PageNumber": "$PageNumber",
"CursorPos": "$CursorPos",
"UserId": "$UserId",
"UserName": "$UserName",
"NotCoverPage": "$NotCoverPage",
"TemplateName": "$TemplateName",
"$
"$$": "$$",
"CurrentDay": "$CurrentDay",
"CurrentMonth": "$CurrentMonth",
"CurrentYear": "$CurrentYear",
"CurrentSecond": "$CurrentSecond",
"CurrentMinute": "$CurrentMinute",
"